In [2]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client
import warnings
warnings.filterwarnings('ignore')
import dask 
import gc
import os
import pandas as pd
import time
client = Client()
client

Client Scheduler: tcp://127.0.0.1:64970 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 17.18 GB


In [3]:
import numpy as np
def read_file(filename:str)->pd.DataFrame:
    return pd.read_csv(filename,error_bad_lines=False,engine='python')


In [4]:
st = time.perf_counter()
foundri_data = read_file('/Users/muhammadaqib/Documents/repos/privaci-rating/data_export/data/population/common/foundri_vendor_cookies.csv')
time.perf_counter()-st

13.549652285000093

In [5]:
len(foundri_data)

890611

In [6]:
foundri_data.columns

Index(['site_name', 'host', 'cookie_name', 'cookie_host', 'is_first_party',
       'path', 'expiry', 'category', 'is_http_only', 'is_session_cookie',
       'is_secure', 'description', 'data_location', 'source_url', 'type',
       'subcategory', 'is_regex', 'country', 'service', 'policy_url',
       'display_name'],
      dtype='object')

In [7]:
res =foundri_data["description"].str.len().max()

In [8]:
res

1130.0

In [11]:
foundri_data.category.value_counts()

Advertising                    387649
Performance & Functionality    296691
Essential                      100874
Unknown                         68759
Analytics & Customization       36638
Name: category, dtype: int64

In [11]:
unique_descriptions = foundri_data.description.value_counts()

In [12]:
unique_descriptions.head(5)

Unknown                                                                                                                                                                                                                                                                                                                                                                                                                              343951
This is an advertising cookie from Casale Media. Casale Media is a tech company providing online display and advertising services to optimize the value of premium content for brands.                                                                                                                                                                                                                                                51745
These may be set through our site by our advertising partners. They may be used by those companies to build a profile of your interests and show

In [103]:
length_condition = foundri_data["description"].str.len() <10 & foundri_data["description"] != ''
short_len_df = foundri_data[length_condition]
short_len_df["description"].replace({"Unknown": "", "unknown": "","uk":""}, inplace=True)

# short_len_df['description'] = short_len_df['description'].map({'Unknown':'','unknown':'','uk':''})



In [105]:
# short_len_df.to_csv('short_length.csv')
print(len(short_len_df))
length_condition = short_len_df["description"].str.len() <10
short_len_df = short_len_df[length_condition]
short_len_df.columns

short_len_df.to_csv('aaaaa.csv')


344639


In [49]:

foundri_data[foundri_data['cookie_name']=='_gat_global']

,site_name,host,cookie_name,cookie_host,is_first_party,path,expiry,category,is_http_only,is_session_cookie,...,description,data_location,source_url,type,subcategory,is_regex,country,service,policy_url,display_name
1179,Tp-link,gotomeeting.com,_gat_global,gotomeeting.com,True,/,1.599198e+09,Performance & Functionality,False,False,...,_gat_global is an unknown cookie from tp-link,NaN,NaN,http_cookie,NaN,NaN,NaN,Gotomeeting,NaN,NaN
469955,1stoplighting,http://www.1stoplighting.com,_gat_global,.1stoplighting.com,True,NaN,1.600836e+09,Unknown,False,False,...,_gat_global is an unknown cookie from tp-link,NaN,NaN,NaN,NaN,NaN,US,NaN,NaN,NaN
497777,Yelp,yelp.my,_gat_global,yelp.my,True,NaN,1.601468e+09,Unknown,False,False,...,Unknown,NaN,NaN,NaN,NaN,NaN,JP,NaN,NaN,NaN
500251,Technobuffalo,technobuffalo.com,_gat_global,technobuffalo.com,True,NaN,1.601458e+09,Unknown,False,False,...,Unknown,NaN,NaN,NaN,NaN,NaN,US,NaN,NaN,NaN
504108,Imore,imore.com,_gat_global,imore.com,True,NaN,1.601460e+09,Unknown,False,False,...,Unknown,NaN,NaN,NaN,NaN,NaN,US,NaN,NaN,NaN
588639,Livechatinc,livechatinc.com,_gat_global,livechatinc.com,True,NaN,1.607432e+09,Unknown,False,False,...,Unknown,NaN,NaN,NaN,NaN,NaN,US,Livechatinc,NaN,NaN


In [50]:
'This cookie collects anonymized information about how users use this site for analysis purposes.'
foundri_data['description']=foundri_data['description'].apply(str)
foundri_data[foundri_data['description'].str.contains('This cookie set by Vimeo to enable the operation of Vimeo videos embedded in this website.')]

,site_name,host,cookie_name,cookie_host,is_first_party,path,expiry,category,is_http_only,is_session_cookie,...,description,data_location,source_url,type,subcategory,is_regex,country,service,policy_url,display_name


In [63]:
#purpose and cat mismatch
res2=foundri_data[ (foundri_data["category"]!='Unknown') & (foundri_data["description"].str.len() >10) & ((foundri_data["description"].str.contains('Unknown '))|(foundri_data["description"].str.contains('unknown')))]

In [78]:

len(res2)

446

In [65]:
res2.head(10)

,site_name,host,cookie_name,cookie_host,is_first_party,path,expiry,category,is_http_only,is_session_cookie,...,description,data_location,source_url,type,subcategory,is_regex,country,service,policy_url,display_name
1332,Rgs,rgs.ru,indexexchange_sync,adhigh.net,False,/,1628002376,Advertising,False,False,...,indexexchange_sync is an unknown cookie from rgs,NaN,NaN,http_cookie,NaN,False,NaN,Adhigh,NaN,NaN
2411,Rgs,rgs.ru,lentainform_sync,adhigh.net,False,/,1628002376,Advertising,False,False,...,lentainform_sync is an unknown cookie from rgs,NaN,NaN,http_cookie,NaN,False,NaN,Adhigh,NaN,NaN
2416,Rgs,rgs.ru,otm_display_sync,adhigh.net,False,/,1628002376,Advertising,False,False,...,otm_display_sync is an unknown cookie from rgs,NaN,NaN,http_cookie,NaN,False,NaN,Adhigh,NaN,NaN
3199,Ibk,gsnpoint.com,ccsession,gsnpoint.com,False,/,-1,Performance & Functionality,False,True,...,ccsession is an unknown cookie from ibk,NaN,NaN,http_cookie,NaN,False,NaN,Gsnpoint,NaN,NaN
112774,Iovation,iovation.com,visitor_id1692,iovation.io,False,/,1630737225,Advertising,False,False,...,visitor_id1692 is an unknown cookie from iovation,NaN,NaN,http_cookie,NaN,False,NaN,Iovation,NaN,NaN
112775,Iovation,iovation.com,visitor_id1692-hash,iovation.io,False,/,1630737225,Essential,False,False,...,visitor_id1692-hash is an unknown cookie from ...,NaN,NaN,http_cookie,NaN,False,NaN,Iovation,NaN,NaN
113919,Ef,ef.com,tts,ef.com,False,/,1596464203,Performance & Functionality,False,False,...,tts is an unknown cookie from ef,NaN,NaN,http_cookie,NaN,False,NaN,Ef,NaN,NaN
178091,Jomashop,jomashop.com,_vis_opt_exp_141_combi,jomashop.com,False,/,1605103797,Performance & Functionality,False,False,...,_vis_opt_exp_141_combi is an unknown cookie fr...,NaN,NaN,http_cookie,NaN,False,NaN,Jomashop,NaN,NaN
178690,Jpost,jpost.com,prism_25900740,jpost.com,False,/,1599055818,Performance & Functionality,False,False,...,prism_25900740 is an Unknown cookie from jpost,NaN,NaN,http_cookie,NaN,False,NaN,Jpost,NaN,NaN
178729,Prensalibre,prensalibre.com,fi_utm,prensalibre.com,False,/,1596467686,Advertising,False,False,...,fi_utm is an Unknown cookie from prensalibre,NaN,NaN,http_cookie,NaN,False,NaN,Prensalibre,NaN,NaN


In [66]:
res2=res2[['host', 'cookie_name', 'cookie_host','category','description']]

In [67]:
res2.to_csv('notUKAndLenGt10.csv',index=False)

In [32]:
#purpose and cat mismatch
res3=foundri_data.loc[ (foundri_data["category"]=='Advertising')  & (foundri_data["description"].str.contains('advert',case=False)==False)
                     &  (foundri_data["description"].str.contains('ad',case=False)==False) &  (foundri_data["description"].str.contains('market',case=False)==False)]




len(res3)

65416

In [33]:
res3['description'].head(35)

1      Used by the social networking service, LinkedI...
16     This cookie is used to group the user behaviou...
71     These cookies are used to track user behavior ...
72     These cookies are used to track user behavior ...
88     Randomly generated person identifier (used to ...
91              Tracking tc_ptid cookie expiration value
186    This is an analytics cookie from Salesforce. S...
203    The visitor hash cookie contains the account I...
207    The visitor cookie includes a unique visitor I...
286    These are performance cookies which we use to ...
349    This cookie is used by Youtube as part of thei...
414    The visitor hash cookie contains the account I...
415    The visitor cookie includes a unique visitor I...
446    The AMCVS cookie serves as a flag indicating t...
458    The visitor cookie includes a unique visitor I...
461    The visitor hash cookie contains the account I...
472    These are performance cookies which we use to ...
476    These are performance co

In [34]:
res3=res3[['host', 'cookie_name', 'cookie_host','category','description']]

In [35]:
res3.to_csv('mismatch_adv_essential.csv',index=False)

In [61]:
#purpose and cat mismatch
res4=foundri_data[(foundri_data["description"].str.contains('unknown', case=False, na=False)==False)
                  & (foundri_data["description"].str.contains('cookie from', case=False, na=False))
                  &  (foundri_data["description"].str.len()<80)]


len(res4)



2503

In [62]:
res4=res4[['host', 'cookie_name', 'cookie_host','category','description']]
res4.to_csv('case3.csv',index=False)

In [63]:
res4.tail(25)

,host,cookie_name,cookie_host,category,description
474998,http://www.scubastore.com,num_items,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
474999,http://www.scubastore.com,id_cesta,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475000,http://www.scubastore.com,mail_cliente,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475001,http://www.scubastore.com,googleuserid,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475003,http://www.scubastore.com,barra_cookies,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475004,http://www.scubastore.com,id_pais,www.scubastore.com,Advertising,id_pais is an Advertising cookie from dressinn
475005,http://www.scubastore.com,id_tienda,www.scubastore.com,Advertising,id_tienda is an Advertising cookie from dressinn
475006,http://www.scubastore.com,gclid_ds,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475007,http://www.scubastore.com,change_domain,www.scubastore.com,Advertising,This is an advertising cookie from tradeinn. T...
475342,http://mimio.com,geot_rocket_country,mimio.boxlight.com,Advertising,geot_rocket_country is an Advertising cookie f...


In [31]:
#purpose and cat mismatch
res5=foundri_data[(foundri_data["category"]=='Unknown') & (foundri_data["description"].str.len()>70) ]


len(res5)



0

In [25]:
res5.description.head(25)

Series([], Name: description, dtype: object)

In [26]:
res5=res5[['host', 'cookie_name', 'cookie_host','category','description']]
res5.to_csv('case4.csv',index=False)

# check duplication

In [67]:
foundri_data['cookie_name'] = foundri_data['cookie_name'].apply(str)
foundri_data['cookie_name'] = foundri_data['cookie_name'].str.strip()
foundri_data = foundri_data.replace(np.nan, '', regex=True)

In [68]:
dup_res = foundri_data[(foundri_data[foundri_data.columns].duplicated()==True) 
                       ]

In [69]:
dup_res.head(2)

,site_name,host,cookie_name,cookie_host,is_first_party,path,expiry,category,is_http_only,is_session_cookie,...,description,data_location,source_url,type,subcategory,is_regex,country,service,policy_url,display_name


In [70]:
dup_res.shape

(0, 21)

In [71]:
dup_res.shape

(0, 21)

In [72]:
li = dup_res.values.tolist()
len(li)

0

In [76]:
li[0]

IndexError: list index out of range

In [77]:
set(li)

set()

In [78]:
dup_res.to_csv('duplicate3.csv',index=False)

In [87]:
new_df_list=[
    ['Cookie_Categiry','Cookie_Purpose'],
    ['Advertising',' This is an advertising cookie '],
    ['Advertising','this is an advertising cookie']
]

In [88]:
new_df_dict= [
    {'Cookie_Categiry': 'Advertising','Cookie_Purpose': ' This is an advertising cookie '},
    {'Cookie_Categiry': 'Advertising','Cookie_Purpose':'this is an advertising cookie'}
]

In [92]:
df_dict=pd.DataFrame(new_df_dict)
df_list=pd.DataFrame(new_df_list[1:],columns=new_df_list[0])

In [90]:
df_dict

,Cookie_Categiry,Cookie_Purpose
0,Advertising,This is an advertising cookie
1,Advertising,this is an advertising cookie


In [93]:
df_list

,Cookie_Categiry,Cookie_Purpose
0,Advertising,This is an advertising cookie
1,Advertising,this is an advertising cookie


In [70]:

new_df_dict= [
    {'Cookie_Categiry': 'Advertising','Cookie_Purpose': ' This is an advertising cookie '},
    {'Cookie_Categiry': 'Advertising','Cookie_Purpose':'this is an advertising cookie'}
]
df_dict=pd.DataFrame(new_df_dict)


In [72]:


df_dict['Cookie_Categiry'] = df_dict['Cookie_Categiry'].map({'Advertising':''})

In [75]:
df_dict.to_csv('checkkk.csv',index=False)

In [20]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})

In [21]:
df

,A,B,C
0,0,5,a
1,1,6,b
2,2,7,c
3,3,8,d
4,4,9,e


In [39]:
df['B']=df['B'].replace(6,'')

In [41]:
df.to_csv('check_it.csv')

In [42]:
df

,A,B,C
0,100,5,A
1,1,,B
2,2,7,c
3,3,8,d
4,4,9,e
